<a href="https://colab.research.google.com/github/ahanna13/MS_NN/blob/main/MS_NN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is my first trial at a neural net for my dataset from RKI:

### Version 4.1 (20230306) of the MALDI-ToF Mass Spectrometry Database for Identification and Classification of Highly Pathogenic Microorganisms from the Robert Koch-Institute (RKI). It can be found at https://zenodo.org/records/7990990